In [1]:
import nivapy3 as nivapy
import pandas as pd
import teotil2 as teo
import useful_rid_code as rid

# Generate overall summary table - 2020

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway and for the four OSPAR regions. This table no longer appears in recent reports, but I find it to be a useful summary document. This notebook reads the output from the data processing so far and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

In [2]:
# Year of interest
year = 2020

## 1. Summarise monitoring data

### 1.1. Identify network nodes with monitoring data

In [3]:
# Read station data
in_xlsx = r"../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=["ospar_region", "nve_vassdrag_nr"])

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network
ann_input_csv = f"../../../teotil2/data/norway_annual_input_data/input_data_{year}.csv"
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

### 1.2. Read monitored fluxes

In [4]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
# mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "new_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("new_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "new_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"]

# Rename cols to match template
mon_df["Flow rate_1000m3/day"] = mon_df["mean_q_1000m3/day"]
del mon_df["mean_q_1000m3/day"]

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr     Cu    Hg   NH4-N  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135        0.0   1.0  0.0   1.0    7.0   7.0    97.0   
                    rid_20         0.0   2.0  0.0   5.0   36.0  10.0    71.0   
NORTH SEA           rid_135        0.0   3.0  0.0   3.0   19.0  27.0   302.0   
                    rid_20         0.0   1.0  0.0   0.0    2.0   1.0    21.0   
NORWEGIAN SEA2      rid_135        0.0   5.0  0.0  11.0   35.0  36.0   384.0   
                    rid_20         0.0   2.0  0.0   3.0   16.0   0.0    30.0   
SKAGERAK            rid_135        0.0   3.0  0.0   2.0   11.0  15.0   232.0   
                    rid_20         0.0  13.0  1.0  18.0   68.0  77.0   494.0   
NORWAY              rid_135        0.0  12.0  1.0  17.0   72.0  84.0  1015.0   
                    rid_20         0.0  17.0  1.0  26.0  122.0  88.0   616.0   

                                     NO3-N    Ni  PO4-P    Pb       SPM  \
ospar_region        new_rid_group                                         
LOFOTEN-BARENTS SEA rid_135          350.0   6.0   18.0   0.0   17380.0   
                    rid_20           821.0  36.0  109.0   1.0  102180.0   
NORTH SEA           rid_135         6050.0  10.0   68.0   7.0   58434.0   
                    rid_20          1132.0   2.0   18.0   1.0    7213.0   
NORWEGIAN SEA2      rid_135         3280.0  25.0  115.0   3.0  147701.0   
                    rid_20          1016.0   7.0   43.0   1.0   84063.0   
SKAGERAK            rid_135         3355.0   6.0   39.0   5.0   33092.0   
                    rid_20         17702.0  38.0  540.0  19.0  454200.0   
NORWAY              rid_135        13035.0  47.0  240.0  15.0  256607.0   
                    rid_20         20670.0  81.0  710.0  21.0  647656.0   

                                       SiO2       TOC     TOTN    TOTP     Zn  \
ospar_region        new_rid_group                                               
LOFOTEN-BARENTS SEA rid_135         31209.0   29412.0   1500.0    32.0    8.0   
                    rid_20         105811.0  102256.0   4100.0   209.0   20.0   
NORTH SEA           rid_135         55371.0   66863.0  10654.0   200.0   72.0   
                    rid_20           8302.0   11582.0   1718.0    48.0    9.0   
NORWEGIAN SEA2      rid_135         89262.0  106276.0   9783.0   266.0   85.0   
                    rid_20          30365.0   33806.0   2009.0    70.0   25.0   
SKAGERAK            rid_135         34290.0   72272.0   6739.0   145.0   64.0   
                    rid_20         210300.0  263905.0  27139.0   884.0  193.0   
NORWAY              rid_135        210132.0  274823.0  28677.0   643.0  230.0   
                    rid_20         354778.0  411549.0  34966.0  1210.0  246.0   

                                   Flow rate  
ospar_region        new_rid_group             
LOFOTEN-BARENTS SEA rid_135          38848.0  
                    rid_20           69600.0  
NORTH SEA           rid_135         127596.0  
                    rid_20           19117.0  
NORWEGIAN SEA2      rid_135         142141.0  
                    rid_20           39839.0  
SKAGERAK            rid_135          40283.0  
                    rid_20          178016.0  
NORWAY              rid_135         348868.0  
                    rid_20          306572.0

## 2. Data for unmonitored areas

In [5]:
# Read data
in_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,478459.0,13958.0,1101.0,2575.0,237.0,68506.0,11813.0,39543.0,816.0,661.0,...,14475.0,0.0,0.0,0.0,6.0,16.0,2.0,0.0,6.0,1306.0
LOFOTEN-BARENTS SEA,148308.0,1515.0,168.0,88.0,7.0,17628.0,3046.0,6670.0,124.0,101.0,...,2432.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,337.0
NORTH SEA,143679.0,4025.0,460.0,494.0,81.0,21552.0,3707.0,14826.0,223.0,276.0,...,5079.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,1.0,410.0
NORWEGIAN SEA2,174488.0,2853.0,360.0,1083.0,111.0,29309.0,5057.0,14684.0,351.0,216.0,...,4034.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,559.0
SKAGERAK,11984.0,5565.0,113.0,910.0,37.0,16.0,3.0,3363.0,118.0,68.0,...,2930.0,0.0,0.0,0.0,4.0,9.0,1.0,0.0,1.0,0.0


## 3. Create table

The cell below uses the 3-fold division of sites (`'rid_11'` etc.) from the old progreamme structure. To use this, copy the old Word template and make sure that the aggregation in Section 1.2 is done by `'old_rid_group'`.

In [6]:
## Create table 3 (old programme structure)
# in_docx = f'../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx'
#
# rid.write_word_overall_table(mon_df, umon_df, in_docx)

The cell below uses the 2-fold division of sites (`'rid_20'` and `'rid_135'`) for the 2017-20 progreamme structure. To use this, copy the new Word template and make sure that the aggregation in Section 1.2 is done by `'new_rid_group'`.

In [7]:
# Create table 3
in_docx = (
    f"../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx"
)
rid.write_word_overall_table_2017_20(mon_df, umon_df, in_docx)

Finished.
